In [ ]:
import numpy as np
from numpy.linalg import norm
from scipy.linalg import fractional_matrix_power as fmp
import matplotlib.pyplot as plt
import pandas as pd
import os
from joblib import Parallel, delayed

In [ ]:
np.random.seed(100)

d = 70

A = np.random.randn(d, d)
A /= norm(A, ord=2)

W = A.T @ A

In [ ]:
def training_set(n, tau, alpha):
    z_train = np.random.randn(d, n)
    z_train /= np.linalg.norm(z_train, axis=0, ord=2)
    for i in range(z_train.shape[0]):    
        z_train[i] = z_train[i] *  (np.random.rand(1) ** (1 / d))
    x_train = fmp(W, alpha) @ z_train
    y_train = A @ x_train + np.random.normal(0, tau, size=(d, n))

    return x_train, y_train

In [ ]:
# Functions

# Tikhonov filter
def tikh(y, la):
    return np.linalg.inv(W + la * np.eye(d)) @ (A.T @ y)


# Landweber filter:
def landweber(y, t):
    landw = np.zeros(d)
    iterates = np.zeros((t, d))
    for i in range(t):
        landw -= .2 * A.T @ (A @ landw - y)
        iterates[i] = landw.copy()
        norm_it =norm(iterates[i], ord=2)
        iterates[i] = iterates[i] / norm_it if norm_it > 1 else iterates[i]
    return landw, iterates


def train_tik(x_tr, y_tr, la):    
    errors = []
    ftik = tikh(y_tr, la)
    mask = np.linalg.norm(ftik, axis=0, ord=2) > 1
    ftik[:, mask] /= np.linalg.norm(ftik[:, mask], axis=0, ord=2)
    err = np.mean(np.linalg.norm(x_tr - ftik, axis=0, ord=2)**2)
    return err

def train_landw(x_tr, y_tr, max_t):
    errors = np.empty((x_tr.shape[1], max_t))
    for i in range(x_tr.shape[1]):
        _, iterates = landweber(y_tr[:, i], max_t)                  
        err = norm(iterates - x_tr[:, i], axis = 1, ord=2) ** 2
        errors[i, :] = err 
    return np.mean(errors, axis=0)


# Functions to get optimal params

def get_lambda(x_tr, y_tr, la): # la is a vector here
    tik_err = np.array([train_tik(x_tr, y_tr, l) for l in la])
    min_index = np.argmin(tik_err)
    return la[min_index], tik_err.min()

def get_t(x_tr, y_tr, iter):  # t is a vector here
    lan_err = train_landw(x_tr, y_tr, iter[-1])
    land_err = lan_err[iter - 1]
    min_index = np.argmin(land_err)
    return iter[min_index], land_err.min()


In [ ]:
# Figure 1

smoothness_level = 0.5
var = .01 # noise level
N_train = 50

x_Train, y_Train = training_set(N_train, var, smoothness_level)

lamb = np.logspace(-4, 2, num=500)

tik_err = []
for i in range(len(lamb)):
    tik_err.append(train_tik(x_Train, y_Train, lamb[i]))

t_emR = np.logspace(- np.log10(1000.5), 0, num=500)
for i in range(len(t_emR)):
    t_emR[i]= int(1 / t_emR[i])
t = t_emR[::-1].astype(int)

lan_err = train_landw(x_Train, y_Train, t[-1])
land_err = lan_err[t - 1]


In [ ]:
# Plot

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5), dpi=300)
ax1.semilogx(lamb, tik_err, label='Tikhonov') 
ax1.set_xlabel(r'$\lambda$', fontsize=25)
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.set_xscale('log')
#ax1.set_yscale('log')
ax1.legend(fontsize=20)


ax2.semilogx(t, land_err, label='Landweber')
ax2.set_xlabel('k', fontsize=25)
ax2.xaxis.set_tick_params(labelsize=20)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.set_xscale('log')
#ax2.set_yscale('log')
ax2.legend(fontsize=20)

position = ax2.get_position()
ax2.set_position([position.x0 - 0.01, position.y0, position.width, position.height])

supylabel = fig.supylabel(r'$\widehat{L}(X_\lambda)$', fontsize=25)
supylabel.set_position((0, 0.5)) 
plt.tight_layout()

plt.savefig("./fig1.pdf", bbox_inches='tight')
plt.show(block=False)

In [ ]:
# Figure 2

def noise_rate (noise, alph):
    rate = (- 4 * alph) / (2 * alph + 1)
    return noise ** rate

n_tr = 1000
tau = np.logspace(-4, -1, num=30)
#Tikhonov
lamb = np.logspace(-5, 0, num=1000) 


# Landweber
t = np.arange(1, 10001)

lamstartik05 = np.zeros(len(tau))
Lstartik05 = np.zeros(len(tau))
lamstartik1 = np.zeros(len(tau))
Lstartik1 = np.zeros(len(tau))

tstarland05 = np.zeros(len(tau))
tstarland1 = np.zeros(len(tau))
Lstarland05 = np.zeros(len(tau))
Lstarland1 = np.zeros(len(tau))

for i in range(len(tau)):
    print('tau=', tau[i])
    x_05, y_05 = training_set(n_tr, tau[i], .5)
    x_1, y_1 = training_set(n_tr, tau[i], 1)
    lamstartik05[i], Lstartik05[i] = get_lambda(x_05, y_05, lamb)
    lamstartik1[i], Lstartik1[i] = get_lambda(x_1, y_1, lamb)
    tstarland05[i], Lstarland05[i] = get_t(x_05, y_05, t)
    tstarland1[i], Lstarland1[i] = get_t(x_1, y_1, t)
    
ftik05 = Lstartik05 * noise_rate(tau, .5)
ftik1 = Lstartik1 * noise_rate(tau, 1.0)
fland05 = Lstarland05 * noise_rate(tau, .5)
fland1 = Lstarland1 * noise_rate(tau, 1.0)

In [ ]:
# Plot

plt.close('all')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5), dpi=300)

ax1.plot(tau, ftik05, linestyle='-', label=r'$s=0.5$')
ax1.plot(tau, ftik1, linestyle='-', label=r'$s=1$')
ax1.legend(fontsize=20)
ax1.set_xscale('log')
#ax1.set_yscale('log')
ax1.tick_params(axis='both', which='major', labelsize=20)
ax1.tick_params(axis='both', which='minor', labelsize=20)
ax1.set_title("Tikhonov", fontsize=25)

ax2.plot(tau, fland05, linestyle='--', label=r'$s=0.5$')
ax2.plot(tau, fland1, linestyle='--', label=r'$s=1$')
ax2.legend(fontsize=20)
ax2.set_xscale('log')
#ax2.set_yscale('log')
ax2.tick_params(axis='both', which='major', labelsize=20)
ax2.tick_params(axis='both', which='minor', labelsize=20)
ax2.set_title("Landweber", fontsize=25)

supylabel = fig.supylabel(r'$L(X_{\lambda_*(\tau)})/\tau^{T}$', fontsize=25)
supylabel.set_position((0, 0.5)) 
fig.supxlabel(r"$\tau$", fontsize=25)

plt.tight_layout()
plt.savefig("./fig2.pdf", bbox_inches='tight')
plt.show(block='False')

In [ ]:
# Figure 3

n_it = 30
n_tr = 10
tau = np.logspace(-4, -1, num=30)

# Tikhonov
lamb = np.logspace(-5, 1, num=500)

#landweber
t_emR = np.logspace(- np.log10(10000.5), 0, num=5000) 
for i in range(len(t_emR)):
    t_emR[i]= int(1 / t_emR[i])
t_emR = t_emR[::-1].astype(int) 

lamb_matrix05 = np.zeros((len(tau), n_it))
lamb_matrix1 = np.zeros((len(tau), n_it))
t_matrix05 = np.zeros((len(tau), n_it))
t_matrix1 = np.zeros((len(tau), n_it))
import time 
for i in range(len(tau)):
    print('tau=', tau[i])
    for j in range(n_it):
        x_05, y_05 = training_set(n_tr, tau[i], .5)
        x_1, y_1 = training_set(n_tr, tau[i], 1)
        lamb_matrix05[i, j] = get_lambda(x_05, y_05, lamb)[0]
        lamb_matrix1[i, j] = get_lambda(x_1, y_1, lamb)[0]
        t_matrix05[i, j] = get_t(x_05, y_05, t_emR)[0]
        t_matrix1[i, j] = get_t(x_1, y_1, t_emR)[0]


In [ ]:
# Plot

dftik05 = pd.DataFrame(lamb_matrix05)
meantik05 = dftik05.mean(axis='columns')
lowertik05 = np.quantile(lamb_matrix05, 0.05, axis=1)
uppertik05 = np.quantile(lamb_matrix05, 0.95, axis=1)

dftik1 = pd.DataFrame(lamb_matrix1)
meantik1 = dftik1.mean(axis='columns')
lowertik1 = np.quantile(lamb_matrix1, 0.05, axis=1)
uppertik1 = np.quantile(lamb_matrix1, 0.95, axis=1)

dfland05 = pd.DataFrame(t_matrix05)
meanland05 = dfland05.mean(axis='columns')
lowerland05 = np.quantile(t_matrix05, 0.05, axis=1)
upperland05 = np.quantile(t_matrix05, 0.95, axis=1)

dfland1 = pd.DataFrame(t_matrix1)
meanland1 = dfland1.mean(axis='columns')
lowerland1 = np.quantile(t_matrix1, 0.05, axis=1)
upperland1 = np.quantile(t_matrix1, 0.95, axis=1)

plt.close('all')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5), dpi=100)

ax1.plot(tau, meantik05, '-', label='$s=0.5$')
ax1.fill_between(tau, lowertik05, uppertik05, alpha=0.2) 
ax1.plot(tau, meantik1, '-', label='$s=1$')
ax1.fill_between(tau, lowertik1, uppertik1, alpha=0.2) 
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.set_yscale('log')
ax1.set_xscale('log')
ax1.set_ylabel(r'$\widehat{\lambda}(\tau)$', fontsize=25)
ax1.legend(fontsize=20)
ax1.set_title("Tikhonov", fontsize=25)

ax2.plot(tau, meanland05, linestyle='--', label='$s=0.5$')
ax2.fill_between(tau, lowerland05, upperland05, alpha=0.2)
ax2.plot(tau, meanland1, linestyle='--', label='$s=1$')
ax2.fill_between(tau, lowerland1, upperland1, alpha=0.2)
ax2.xaxis.set_tick_params(labelsize=20)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.set_yscale('log')
ax2.set_xscale('log')
ax2.set_ylabel(r'$\widehat{k}(\tau)$', fontsize=25)
ax2.legend(fontsize=20)
ax2.set_title("Landweber", fontsize=25)
fig.supxlabel(r'$\tau$', fontsize=25)
plt.tight_layout(pad=2, w_pad=1.2, h_pad=0.5)
plt.savefig("./fig3.pdf", bbox_inches='tight')
plt.show(block=False)

In [ ]:
# Figure 4

smoothness_level = .5
var = 0.01 # noise level
N_train = 1000
x_Train, y_Train = training_set(N_train, var, smoothness_level)
n_it = 30

lamb = np.logspace(-4, 0, num=500)

t_emR = np.logspace(- np.log10(1000.5), 0, num=500) #max_t=500 even for tau=0.01, alph=.5
for i in range(len(t_emR)):
    t_emR[i]= int(1 / t_emR[i])
    
t1 = t_emR[::-1].astype(int)

N_vec = np.arange(5, 101, 5)

L_at_lhatn = np.zeros((len(N_vec), n_it))
L_at_thatn = np.zeros((len(N_vec), n_it))
lamb_hatn = np.zeros((len(N_vec), n_it))
t_hatn = np.zeros((len(N_vec), n_it))

for i in range(len(N_vec)):
    print('n=', N_vec[i])
    for j in range(n_it):
        xsmall, ysmall = training_set(N_vec[i], var, smoothness_level)
        
        lamb_hatn[i, j] = get_lambda(xsmall, ysmall, lamb)[0]
        L_at_lhatn[i, j] = train_tik(x_Train, y_Train, lamb_hatn[i, j])
        
        t_hatn[i, j] = get_t(xsmall, ysmall, t1)[0]
        L_at_thatn[i, j] = train_landw(x_Train, y_Train, int(t_hatn[i, j]))[-1]


In [ ]:
# Plot

meantik = np.mean(L_at_lhatn, axis=1) 
lowertik = np.quantile(L_at_lhatn, 0.05, axis=1) 
uppertik = np.quantile(L_at_lhatn, 0.95, axis=1) 

meanland = np.mean(L_at_thatn, axis=1)
lowerland = np.quantile(L_at_thatn, 0.05, axis=1) 
upperland = np.quantile(L_at_thatn, 0.95, axis=1)


plt.close('all')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5), dpi=100)

ax1.plot(N_vec, meantik, '-', label='Tikhonov')
ax1.scatter(N_vec, meantik, color='red', s=50) 
ax1.fill_between(N_vec, lowertik, uppertik, alpha=0.2)
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
#ax1.set_yscale('log')
ax1.set_xscale('log')
ax1.legend(fontsize=20)

ax2.plot(N_vec, meanland, linestyle='-', label='Landweber')
ax2.scatter(N_vec, meanland, color='red', s=50) 
ax2.fill_between(N_vec, lowerland, upperland, alpha=0.2)
#ax2.set_yscale('log')
ax2.set_xscale('log')
ax2.xaxis.set_tick_params(labelsize=20)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.legend(fontsize=20)

fig.supxlabel(r'$n$', fontsize=25)
fig.supylabel(r'$L(X_{\widehat{\lambda}(n)})$', fontsize=25)
plt.tight_layout(pad=2, w_pad=1.2, h_pad=0.8)
plt.savefig("./fig4.pdf", bbox_inches='tight')  # This shud go before show
plt.show(block=False)

In [ ]:
# Figure 5

smalln = 5
bign = 500
alpha = .5
n_it = 30
tau = np.logspace(-4, -1, 30)

#Tikhonov
lamb1 = np.logspace(-5, 0, num=2000)
lamb2 = np.logspace(-5, 0, num=500)

# Landweber
t1 = np.arange(1, 10001)

t_emR2 = np.logspace(- np.log10(10000.5), 0, num=3000)
for i in range(len(t_emR2)):
    t_emR2[i]= int(1 / t_emR2[i])

t2 = t_emR2[::-1].astype(int)

In [ ]:
lamstartik = np.zeros(len(tau))
Lstartik = np.zeros(len(tau))
Lhattik = np.zeros((len(tau), n_it))
lamb_hat = np.zeros((len(tau), n_it))

tstarlan = np.zeros(len(tau))
Lstarlan = np.zeros(len(tau))
Lhatlan = np.zeros((len(tau), n_it))
t_hat = np.zeros((len(tau), n_it))

for i in range(len(tau)):
    print('i=', i)
    x_big, y_big = training_set(bign, tau[i], alpha)

    lamstartik[i], Lstartik[i] = get_lambda(x_big, y_big, lamb1)
    tstarlan[i], Lstarlan[i] = get_t(x_big, y_big, t1)
    for j in range(n_it):
        x_small, y_small = training_set(smalln, tau[i], alpha)
        #T
        lamb_hat[i, j] = get_lambda(x_small, y_small, lamb2)[0]
        Lhattik[i, j] = train_tik(x_big, y_big, lamb_hat[i, j])
        #L
        t_hat[i, j] = get_t(x_small, y_small, t2)[0]
        Lhatlan[i, j] = train_landw(x_big, y_big, int(t_hat[i, j]))[-1]


In [ ]:
# QO criterion

# Cross-validation functions

# Tikhonov
def q_get_lambda_hat(lamb, var, alph, n):
    x_tr, y_tr = training_set(n, var, alph)
    tik_err = np.array([train_tik(x_tr, y_tr, l) for l in lamb])
    return lamb[np.argmin(tik_err)]


# Landweber
def q_get_t_hat(t, var, alph, n):
    x_tr, y_tr = training_set(n, var, alph)
    errors = train_landw(x_tr, y_tr, t[-1])
    land_err = errors[t - 1]
    return t[np.argmin(land_err)]


#QO functions
def qopt_tikhonov(y, lamda):
    tiks = np.array([tikh(y, l) for l in lamda])
    tikdif = np.array([norm(tiks[i+1] - tiks[i]) for i in range(len(tiks) - 1)])
    pos = np.argmin(tikdif)
    return lamda[pos]


def qopt_landweber(y, t):
    landwdif = np.zeros(len(t))
    iterates = landweber(y, len(t))[1]
    for i in range(len(landwdif)):
        landwdif[i] = norm(iterates[i] - landweber(A @ iterates[i], i+1)[0])
    pos = np.argmin(landwdif)
    return t[pos]


In [ ]:
std = 1e-2  # or np.sqrt(1e-3), 0.1, 1
alpha=.5

N_test = 50
x_test, y_test = training_set(N_test, std, alpha)

lamb = np.logspace(-5, 1, num=500)

t_emR = np.logspace(- np.log10(500.5), 0, num=50)
for i in range(len(t_emR)):
    t_emR[i]= int(1 / t_emR[i])
    
t = t_emR[::-1].astype(int)

n_it = 30

#Best parameters for each case and errors

N_train = 1000
lamb_hat_opt = np.array([q_get_lambda_hat(lamb, std, alpha, N_train) for i in range(n_it)])
t_hat_opt = np.array([q_get_t_hat(t, std, alpha, N_train) for i in range(n_it)])
t_hat_opt = t_hat_opt.astype(int)

crosstik = np.array([train_tik(x_test, y_test, l) for l in lamb_hat_opt])
crossland = np.array([train_landw(x_test, y_test, l) for l in t_hat_opt])

qopt_tik = np.array([qopt_tikhonov(y, lamb) for y in y_test.T])
qopt_land = np.array([qopt_landweber(y, lamb) for y in y_test.T])

f_qopt_tik = np.zeros((N_test, d))
f_qopt_land = np.zeros((N_test, d))
for i in range(N_test):
    f_qopt_tik[i] = tikh(y_test.T[i], qopt_tik[i])
    f_qopt_land[i] = landweber(y_test.T[i], int(qopt_land[i]))[0]

err_qopt_tik = np.mean(norm(f_qopt_tik - x_test.T, axis=1) ** 2, axis=0)
err_qopt_land = np.mean(norm(f_qopt_land - x_test.T, axis=1) ** 2, axis=0)

meantikqopt = np.mean(crosstik - err_qopt_tik)
vartikqopt = np.std(crosstik - err_qopt_tik)
meanlandqopt = np.mean(crossland - err_qopt_land)
varlandqopt = np.std(crossland - err_qopt_land)

In [ ]:
print('meantikqopt=', meantikqopt)
print('vartqopt=', vartikqopt)
print('meanlanqopt=', meanlandqopt)
print('varlandqopt=', varlandqopt)